In [50]:
#Uses python3

import numpy as np
import pandas as pd 
import time
import matplotlib
import matplotlib.pyplot as plt
import sys

In [115]:
def coord(site):
    """get coordinate i of vector"""
    x = site // L
    y = site - x*L
    return (x,y)

def get(i):
    """fixin' boundary"""
    if i<0: return i
    else: return i % L
    
def get_neigh():
    """get neighbour's arr"""
    s = np.arange(L**2).reshape(L,L)
    nei = []
    for site in range(L*L):
        i,j = coord(site)
        nei += [s[get(i-1),get(j-1)],s[get(i-1),get(j)],s[get(i),get(j+1)],
                s[get(i+1),get(j+1)],s[get(i+1),get(j)],s[get(i),get(j-1)]]
    return np.array(nei, dtype=np.int32).reshape(L*L,6)

#=============================================================================

def gen_state():
    """generate random init. state with lenght L*L and q=[-1,1]"""
    return np.array([np.random.choice([-1,1]) for _ in range(L*L)], dtype=np.int32)

def mc_choice(dE,T):
    """принимаем или не принимаем переворот спина?"""
    if dE > 0:
        return True
    elif np.random.uniform() <= np.exp(2*dE/T): 
        return True
    else:
        return False

def calc_dE(site, nei):
    """calculate dE"""
    summa = (
        nei[0]*nei[1]+nei[1]*nei[2]+
        nei[2]*nei[3]+nei[3]*nei[4]+
        nei[4]*nei[5]+nei[5]*nei[0]
    )
    return -site*summa
    
def step(s,nei,T):
    """крутим 1 спин"""
    j,k = np.random.randint(-1,L-1), np.random.randint(-1,L-1) 
    if j==-1: j=L-1
    if k==-1: k=L-1
    site = j*L+k
    
    neigh = s[nei[site,:]]               
    dE = calc_dE(s[site], neigh)
    if mc_choice(dE,T):
        s[site] *= -1


def mc_step(s,nei,T):
    """perform L*L flips for 1 MC step"""
    for _ in range(L*L):
        step(s,nei,T)
        
        
def model(T,N_avg=10,N_mc=10,Relax=10):
    """Моделируем АТ"""
    E, M = [], []

    state = gen_state()
    nei = get_neigh()
    
    #relax $Relax times be4 AVG
    for __ in range(Relax):
            mc_step(state, nei, T)
    #AVG every $N_mc steps
    for _ in range(N_avg):
        for __ in range(N_mc):
            mc_step(state, nei, T)
        E += [calc_e(state)]
        M += [calc_m(state)]
    
    return E, M
        
#===========================================================================================

def create_mask():
    """маска в виде 3 под-решёток"""
    a = np.asarray([i % 3 for i in range(L)])
    return (a + a[:, None])%3

def calc_e(st):
    st = st.reshape(L,L)
    """calculate energy per site
        # expland state matrix"""
    a = np.concatenate((st[L-1].reshape(1,L), st, st[0].reshape(1,L)), axis=0) 
    b = np.concatenate((a[:,-1].reshape(L+2,1),a,a[:,0].reshape(L+2,1)), axis=1)
    return -np.sum(b[1:-1, 1:-1]*b[2:, 2:]*(b[2:, 1:-1]+b[1:-1, 2:]))/(L*L)  

def calc_ms(st):
    """magnetization"""
    st = st.reshape(L,L)
    msr = np.array([np.sum(st[mask==i]) for i in [0,1,2]])/(L*L)
    return np.sqrt(np.sum(msr*msr))



In [118]:
global L, mask
L = 100
mask = create_mask()

np.random.seed(1)
s = gen_state(); nei = get_neigh()

%timeit [mc_step(s, nei, 1) for _ in range(10)]

3.02 s ± 81.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [116]:
global L, mask
L = 100
mask = create_mask()

np.random.seed(1)
s = gen_state(); nei = get_neigh()
print(calc_e(s))

for _ in range(10):
    mc_step(s, nei, 1)
print(calc_e(s))

-0.0048
-1.1


In [61]:
# py     -- 2.16 s ± 52.6 ms per loop     -- original
# py2    -- 3.02 s ± 81.8 ms per loop     -- tabulate neigh
